# 1. Entorno

In [1]:
#Creamos un Notebook en PySpark sobre Athena

# 2. Librerias

In [2]:
# Utilitarios para modificar el esquema de metadatos
from pyspark.sql.types import StructType, StructField

#Importamos los tipos de datos que definiremos para cada campo
from pyspark.sql.types import StringType, IntegerType, DoubleType

#Importamos la librerIa de pandas compatible con entornos de clUster de Big Data
import pyspark.pandas as pd

#Por defecto un dataframe Pandas muestra 1000 registros
#Vamos a indicarle que solo muestre 20 para que no se sature el notebook
pd.set_option("display.max_rows", 20)

#Libreria para manipular los servicios de AWS
import boto3

#Libreria utilitaria para JSON
import json

Calculation started (calculation_id=08c83c65-dd62-0bae-7a51-8138caddb08d) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# 3. Lectura del archivo listo para analizar

In [65]:
#Leemos los datos
#No es necesario indicar el esquema de metadatos, ya que no vamos a procesar los campos
#S�lo vamos a crear los cortes del dataframe para la validaci�n cruzada
#IMPORTANTE: Notar que los archivos ya no tienen header
#IMPORTANTE: Reemplazar "XXX" por tus iniciales
dfDataset = spark.read.format("csv").option("header", "false").option("delimiter", ",").option("encoding", "ISO-8859-1").load("s3://datasetsbdatestXXX/data/insurance_dataset/")

#Vemos el esquema de metadatos
dfDataset.printSchema()

#Verificamos
dfDataset.show()

Calculation started (calculation_id=e8c83c75-a9df-7e31-da91-433207aea76f) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)

+-----------+----+------+---+---+---+---+---+---+---+----+----+
|        _c0| _c1|   _c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|
+-----------+----+------+---+---+---+---+---+---+---+----+----+
|  4837.5823|30.0| 31.57|3.0|  0|  1|  0|  0|  1|  0|   1|   0|
|  3943.5954|29.0| 31.16|0.0|  1|  0|  1|  0|  0|  0|   1|   0|
|   4399.731|36.0|  29.7|0.0|  0|  1|  0|  0|  1|  0|   1|   0|
|  6185.3208|41.0| 31.02|0.0|  1|  0|  0|  0|  1|  0|   1|   0|
| 46200.9851|44.0| 43.89|2.0|  1|  0|  0|  0|  1|  0|   0|   1|
| 7222.78625|45.0|21.375|0.0|  0|  1|

# 4. Division de dataframe en 5 partes

In [66]:
#Utilitarios de Spark
import pyspark.sql.functions as f

Calculation started (calculation_id=d8c83c76-6ba5-4083-83c1-edd25d6d3ace) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [67]:
#Agregamos una columna que indique el �ndice de la fila
dfIndice = dfDataset.withColumn("indice_fila", f.monotonically_increasing_id())

#Verificamos
dfIndice.show()

Calculation started (calculation_id=3ec83c76-70d0-adc1-ff17-7da864c91e7e) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+-----------+----+------+---+---+---+---+---+---+---+----+----+-----------+
|        _c0| _c1|   _c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|indice_fila|
+-----------+----+------+---+---+---+---+---+---+---+----+----+-----------+
|  4837.5823|30.0| 31.57|3.0|  0|  1|  0|  0|  1|  0|   1|   0|          0|
|  3943.5954|29.0| 31.16|0.0|  1|  0|  1|  0|  0|  0|   1|   0|          1|
|   4399.731|36.0|  29.7|0.0|  0|  1|  0|  0|  1|  0|   1|   0|          2|
|  6185.3208|41.0| 31.02|0.0|  1|  0|  0|  0|  1|  0|   1|   0|          3|
| 46200.9851|44.0| 43.89|2.0|  1|  0|  0|  0|  1|  0|   0|   1|          4|
| 7222.78625|45.0|21.375|0.0|  0|  1|  0|  1|  0|  0|   1|   0|          5|
| 12485.8009|55.0| 40.81|3.0|  1|  0|  0|  0|  1|  0|   1|   0|          6|
| 46130.5265|60.0| 31.35|3.0|  0|  1|  0|  1|  0|  0|   0|   1|          7|
|  12363.547|56.0|  36.1|3.0|  0|  1|  0|  0|  0|  1|   1|   0|          8|
| 10156.7832|49.0| 23.18|2.0|  1|  0|  0|  1|  0|  0|   1|   0|  

In [68]:
#Veamos solo el campo "indice_fila" y todos los registros
#Vemos que no neceriamente los �ndices son incrementales
dfIndice.select(dfIndice["indice_fila"]).show(2000, False)

Calculation started (calculation_id=30c83c76-8202-ef5b-8a06-543d720531e4) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+-----------+
|indice_fila|
+-----------+
|0          |
|1          |
|2          |
|3          |
|4          |
|5          |
|6          |
|7          |
|8          |
|9          |
|10         |
|11         |
|12         |
|13         |
|14         |
|15         |
|16         |
|17         |
|18         |
|19         |
|20         |
|21         |
|22         |
|23         |
|24         |
|25         |
|26         |
|27         |
|28         |
|29         |
|30         |
|31         |
|32         |
|33         |
|34         |
|35         |
|36         |
|37         |
|38         |
|39         |
|40         |
|41         |
|42         |
|43         |
|44         |
|45         |
|46         |
|47         |
|48         |
|49         |
|50         |
|51         |
|52         |
|53         |
|54         |
|55         |
|56         |
|57         |
|58         |
|59         |
|60         |
|61         |
|62         |
|63         |
|64         |
|65         |
|66        

In [69]:
#Para solucionar lo anterior usaremos el utilitario "Window"
#Este utilitario permite definir una columna para ordenar los registros
#Los ordenaremos por la columna "indice_fila"
from pyspark.sql.window import Window

#Agregamos la columna "indice_fila_2"
#Usamos nuevamente la funci�n "row_number" para agregar el �ndice
#Pero esta vez se generar� en orden, ya que estamos usando el Window.orderBy
dfIndice = dfIndice.withColumn(
    "indice_fila_2", 
    f.row_number().over(Window.orderBy("indice_fila"))
)

Calculation started (calculation_id=5cc83c76-a295-e0cc-ef50-87e54b263f31) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [70]:
#Verificamos los �ndices
dfIndice.select(dfIndice["indice_fila_2"]).show(2000, False)

Calculation started (calculation_id=a2c83c76-afa0-998d-5089-8b01d7130d9a) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+-------------+
|indice_fila_2|
+-------------+
|1            |
|2            |
|3            |
|4            |
|5            |
|6            |
|7            |
|8            |
|9            |
|10           |
|11           |
|12           |
|13           |
|14           |
|15           |
|16           |
|17           |
|18           |
|19           |
|20           |
|21           |
|22           |
|23           |
|24           |
|25           |
|26           |
|27           |
|28           |
|29           |
|30           |
|31           |
|32           |
|33           |
|34           |
|35           |
|36           |
|37           |
|38           |
|39           |
|40           |
|41           |
|42           |
|43           |
|44           |
|45           |
|46           |
|47           |
|48           |
|49           |
|50           |
|51           |
|52           |
|53           |
|54           |
|55           |
|56           |
|57           |
|58           |
|

In [71]:
#Obtenemos el n�mero total de registros
numeroDeRegistros = dfIndice.count()

#Verificamos
print(numeroDeRegistros)

Calculation started (calculation_id=34c83c76-d2b5-7b20-9039-805ec318f194) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
1338



In [72]:
#Calculamos cu�ntos registros representan el 20%
cantidadDeRegistrosValidacion = int(numeroDeRegistros/5)

#Verificamos
print(cantidadDeRegistrosValidacion)

Calculation started (calculation_id=ccc83c76-dff8-94cc-796a-2e7069c2ea86) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
267



In [73]:
#Obtenemos el PRIMER 20% de registros
df1 = dfIndice.filter(
    (dfIndice["indice_fila_2"] >= 0) &
    (dfIndice["indice_fila_2"] < cantidadDeRegistrosValidacion)
).drop("indice_fila").drop("indice_fila_2")

#Verificamos
df1.show()

Calculation started (calculation_id=3ec83c76-ec60-bfef-0a96-c097b37bc2bc) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+-----------+----+------+---+---+---+---+---+---+---+----+----+
|        _c0| _c1|   _c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|
+-----------+----+------+---+---+---+---+---+---+---+----+----+
|  4837.5823|30.0| 31.57|3.0|  0|  1|  0|  0|  1|  0|   1|   0|
|  3943.5954|29.0| 31.16|0.0|  1|  0|  1|  0|  0|  0|   1|   0|
|   4399.731|36.0|  29.7|0.0|  0|  1|  0|  0|  1|  0|   1|   0|
|  6185.3208|41.0| 31.02|0.0|  1|  0|  0|  0|  1|  0|   1|   0|
| 46200.9851|44.0| 43.89|2.0|  1|  0|  0|  0|  1|  0|   0|   1|
| 7222.78625|45.0|21.375|0.0|  0|  1|  0|  1|  0|  0|   1|   0|
| 12485.8009|55.0| 40.81|3.0|  1|  0|  0|  0|  1|  0|   1|   0|
| 46130.5265|60.0| 31.35|3.0|  0|  1|  0|  1|  0|  0|   0|   1|
|  12363.547|56.0|  36.1|3.0|  0|  1|  0|  0|  0|  1|   1|   0|
| 10156.7832|49.0| 23.18|2.0|  1|  0|  0|  1|  0|  0|   1|   0|
|   2585.269|21.0|  17.4|1.0|  1|  0|  0|  0|  0|  1|   1|   0|
|    1242.26|19.0|  20.3|0.0|  0|  1|  0|  0|  0|  1|   1|   0|
|   40103.89|39.0

In [74]:
#Obtenemos el SEGUNDO 20% de registros
df2 = dfIndice.filter(
    (dfIndice["indice_fila_2"] >= cantidadDeRegistrosValidacion) &
    (dfIndice["indice_fila_2"] < 2*cantidadDeRegistrosValidacion)
).drop("indice_fila").drop("indice_fila_2")

#Verificamos
df2.show()

Calculation started (calculation_id=b6c83c76-fdd6-b928-22fe-28fb5016ea40) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+-----------+----+------+---+---+---+---+---+---+---+----+----+
|        _c0| _c1|   _c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|
+-----------+----+------+---+---+---+---+---+---+---+----+----+
|32108.66282|44.0|29.735|2.0|  0|  1|  1|  0|  0|  0|   1|   0|
| 8965.79575|39.0|24.225|5.0|  1|  0|  0|  1|  0|  0|   1|   0|
|  2304.0022|18.0| 26.18|2.0|  0|  1|  0|  0|  1|  0|   1|   0|
|  9487.6442|53.0| 29.48|0.0|  0|  1|  0|  0|  1|  0|   1|   0|
|  1121.8739|18.0| 23.21|0.0|  0|  1|  0|  0|  1|  0|   1|   0|
|  9549.5651|50.0| 46.09|1.0|  1|  0|  0|  0|  1|  0|   1|   0|
| 2217.46915|18.0|40.185|0.0|  1|  0|  1|  0|  0|  0|   1|   0|
|  1628.4709|19.0| 22.61|0.0|  0|  1|  0|  1|  0|  0|   1|   0|
| 12982.8747|62.0| 39.93|0.0|  0|  1|  0|  0|  1|  0|   1|   0|
|   11674.13|56.0|  35.8|1.0|  1|  0|  0|  0|  0|  1|   1|   0|
|   7160.094|42.0|  35.8|2.0|  0|  1|  0|  0|  0|  1|   1|   0|
|  39047.285|37.0|  34.2|1.0|  0|  1|  1|  0|  0|  0|   0|   1|
| 6358.77645|42.0

In [75]:
#Obtenemos todos los cortes

#PRIMER CORTE
df1 = dfIndice.filter(
    (dfIndice["indice_fila_2"] >= 0) &
    (dfIndice["indice_fila_2"] < cantidadDeRegistrosValidacion)
).drop("indice_fila").drop("indice_fila_2")

#SEGUNDO CORTE
df2 = dfIndice.filter(
    (dfIndice["indice_fila_2"] >= cantidadDeRegistrosValidacion) &
    (dfIndice["indice_fila_2"] < 2*cantidadDeRegistrosValidacion)
).drop("indice_fila").drop("indice_fila_2")

#TERCER CORTE
df3 = dfIndice.filter(
    (dfIndice["indice_fila_2"] >= 2*cantidadDeRegistrosValidacion) &
    (dfIndice["indice_fila_2"] < 3*cantidadDeRegistrosValidacion)
).drop("indice_fila").drop("indice_fila_2")

#CUARTO CORTE
df4 = dfIndice.filter(
    (dfIndice["indice_fila_2"] >= 3*cantidadDeRegistrosValidacion) &
    (dfIndice["indice_fila_2"] < 4*cantidadDeRegistrosValidacion)
).drop("indice_fila").drop("indice_fila_2")

#QUINTO CORTE
df5 = dfIndice.filter(
    (dfIndice["indice_fila_2"] >= 4*cantidadDeRegistrosValidacion) &
    (dfIndice["indice_fila_2"] <= 5*cantidadDeRegistrosValidacion)
).drop("indice_fila").drop("indice_fila_2")

Calculation started (calculation_id=dcc83c77-190f-575a-dbae-1f17637134ec) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# 11. Creaci�n de datasets para validaci�n cruzada

In [76]:
#Nombre del bucket desde donde se lee el archivo
#IMPORTANTE: REEMPLAZAR "XXX" POR TUS INICIALES
bucket = "datasetsbdatestXXX"

Calculation started (calculation_id=8ec83c77-453b-6755-6912-def07e646761) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [77]:
#Directorio del dataset
directorioDataset = "data/insurance_dataset_validacion_cruzada/vc1"

Calculation started (calculation_id=f8c83c77-4667-1b0d-2f91-9a4109068a75) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [78]:
#Directorio en donde est�n los archivos de entrenamiento
directorioDeEntrenamiento = f"s3://{bucket}/{directorioDataset}/train/"

#Verificamos
print(directorioDeEntrenamiento)

Calculation started (calculation_id=8ec83c77-4e87-038d-39ce-53d8b2adb3c2) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
s3://datasetsbdatest001/data/insurance_dataset_validacion_cruzada/vc1/train/



In [79]:
#Directorio en donde est�n los archivos de validaci�n
directorioDeValidacion = f"s3://{bucket}/{directorioDataset}/test/"

#Verificamos
print(directorioDeValidacion)

Calculation started (calculation_id=88c83c77-5445-c26f-fa5e-211cf7922da9) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
s3://datasetsbdatest001/data/insurance_dataset_validacion_cruzada/vc1/test/



In [80]:
#Definimos quien es el dataframe de validaci�n
dfTest = df1

Calculation started (calculation_id=c0c83c77-5969-473e-1def-f46b3fe56897) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [81]:
#El resto de registros ser�n para entrenar
dfTrain = df2.union(df3).union(df4).union(df5)

Calculation started (calculation_id=a2c83c77-5a8f-8b22-32f6-8ef1831be6ba) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [82]:
#Almacenamos el dataframe de entrenamiento
dfTrain.write.format("csv").option("header", "false").option("delimiter", ",").option("encoding", "ISO-8859-1").mode("overwrite").save(directorioDeEntrenamiento)

Calculation started (calculation_id=d6c83c77-61b5-4cd7-2dbb-559f69dc2a88) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [83]:
#Almacenamos el dataframe de Validación
dfTest.write.format("csv").option("header", "false").option("delimiter", ",").option("encoding", "ISO-8859-1").mode("overwrite").save(directorioDeValidacion)

Calculation started (calculation_id=9ec83c77-7719-2905-8553-15693e7112d8) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [57]:
#Nos conectamos al servicio de "S3" para eliminar los archivos "_SUCCESS"
s3 = boto3.client("s3")

Calculation started (calculation_id=7ec83c6f-834d-64e8-cbe1-c3317a530874) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [84]:
#Eliminamos el archivo "_SUCCESS" del dataset de entrenamiento
s3.delete_object(
    Bucket = bucket,
    Key = f"{directorioDataset}/train/_SUCCESS"
)

Calculation started (calculation_id=02c83c77-8c8e-a1a6-fa1e-2649a4e06051) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
{'ResponseMetadata': {'RequestId': '1K0J5SBKSNR5GBMV', 'HostId': 'UggdqBEW6N/3UHMrJ1b8gSbltwi4dhpfwSaNi3pRRspdM96+32bCA3hZE1CydlbM7ZV1u1KroOM=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'UggdqBEW6N/3UHMrJ1b8gSbltwi4dhpfwSaNi3pRRspdM96+32bCA3hZE1CydlbM7ZV1u1KroOM=', 'x-amz-request-id': '1K0J5SBKSNR5GBMV', 'date': 'Wed, 03 Jul 2024 14:10:55 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}



In [85]:
#Eliminamos el archivo "_SUCCESS" del dataset de validaci�n
s3.delete_object(
    Bucket = bucket,
    Key = f"{directorioDataset}/test/_SUCCESS"
)

Calculation started (calculation_id=eec83c77-91a1-9af4-8691-e082061ed806) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
{'ResponseMetadata': {'RequestId': 'TMKPH12387EGZPZ9', 'HostId': 'E1bNl4D/HKIZF9Vb7zzOSkRdVtjGyblZayto4Xr/2CbWyLNe1/2UCVLMGvhgFxGUySA/MevcxqI=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'E1bNl4D/HKIZF9Vb7zzOSkRdVtjGyblZayto4Xr/2CbWyLNe1/2UCVLMGvhgFxGUySA/MevcxqI=', 'x-amz-request-id': 'TMKPH12387EGZPZ9', 'date': 'Wed, 03 Jul 2024 14:10:57 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}



In [86]:

#Funci�n utilitaria
def generar_dataset_validacion_cruzada(dfTest, dfTrain, bucket, directorioDataset):
    #Directorio en donde est�n los archivos de entrenamiento
    directorioDeEntrenamiento = f"s3://{bucket}/{directorioDataset}/train/"
    #Directorio en donde est�n los archivos de validaci�n
    directorioDeValidacion = f"s3://{bucket}/{directorioDataset}/test/"
    
    #Almacenamos el dataframe de entrenamiento
    dfTrain.write.format("csv").option("header", "false").option("delimiter", ",").option("encoding", "ISO-8859-1").mode("overwrite").save(directorioDeEntrenamiento)

    #Almacenamos el dataframe de entrenamiento
    dfTest.write.format("csv").option("header", "false").option("delimiter", ",").option("encoding", "ISO-8859-1").mode("overwrite").save(directorioDeValidacion)
    
    #Nos conectamos al servicio de "S3" para eliminar los archivos "_SUCCESS"
    s3 = boto3.client("s3")

    #Eliminamos el archivo "_SUCCESS" del dataset de entrenamiento
    s3.delete_object(
        Bucket = bucket,
        Key = f"{directorioDataset}/train/_SUCCESS"
    )

    #Eliminamos el archivo "_SUCCESS" del dataset de validaci�n
    s3.delete_object(
        Bucket = bucket,
        Key = f"{directorioDataset}/test/_SUCCESS"
    )

Calculation started (calculation_id=4ac83c77-96bf-9e68-b477-bfbe2af8f1aa) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [87]:
#Generamos el PRIMER conjunto de datasets
generar_dataset_validacion_cruzada(
    df1,
    df2.union(df3).union(df4).union(df5),
    bucket,
    "data/insurance_dataset_validacion_cruzada/vc1"
)

Calculation started (calculation_id=62c83c78-3cf7-7a82-b0fe-e5bba0bafbb9) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [88]:
#Generamos todos los datasets

#Generamos el PRIMER conjunto de datasets
generar_dataset_validacion_cruzada(
    df1,
    df2.union(df3).union(df4).union(df5),
    bucket,
    "data/insurance_dataset_validacion_cruzada/vc1"
)

#Generamos el SEGUNDO conjunto de datasets
generar_dataset_validacion_cruzada(
    df2,
    df1.union(df3).union(df4).union(df5),
    bucket,
    "data/insurance_dataset_validacion_cruzada/vc2"
)

#Generamos el TERCER conjunto de datasets
generar_dataset_validacion_cruzada(
    df3,
    df1.union(df2).union(df4).union(df5),
    bucket,
    "data/insurance_dataset_validacion_cruzada/vc3"
)

#Generamos el CUARTO conjunto de datasets
generar_dataset_validacion_cruzada(
    df4,
    df1.union(df2).union(df3).union(df5),
    bucket,
    "data/insurance_dataset_validacion_cruzada/vc4"
)

#Generamos el QUINTO conjunto de datasets
generar_dataset_validacion_cruzada(
    df5,
    df1.union(df2).union(df3).union(df4),
    bucket,
    "data/insurance_dataset_validacion_cruzada/vc5"
)

Calculation started (calculation_id=66c83c78-dcfc-d8f9-2c0b-380e3cb90a17) in (session=4ac83c64-3dcd-aa7b-edd3-018e0263fd1d). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
